# Mount Drive to Save Model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing Modules

In [2]:
!pip install -qq transformers[sentencepiece]

     |████████████████████████████████| 2.8 MB 11.0 MB/s 
     |████████████████████████████████| 636 kB 48.2 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.7 MB/s 


# Imports

In [3]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import json
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

# Obtaining and Preprocessing Data

In [4]:
%%capture
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [5]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('dev-v2.0.json')

In [6]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

# Initializing Models

In [7]:
query_model_name = "facebook/dpr-question_encoder-single-nq-base"
passage_model_name = "facebook/dpr-ctx_encoder-single-nq-base"

query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(query_model_name)
passage_tokenizer = DPRContextEncoderTokenizer.from_pretrained(passage_model_name)
query_model = DPRQuestionEncoder.from_pretrained(query_model_name)
passage_model = DPRContextEncoder.from_pretrained(passage_model_name)

query_model.train()
passage_model.train()
print()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

# Initializing Tokenizers

In [8]:
train_query_encodings = query_tokenizer(train_questions, truncation=True, padding=True, return_tensors = 'pt')
train_context_encodings = passage_tokenizer(train_contexts, truncation=True, padding=True, return_tensors = 'pt')

val_query_encodings = query_tokenizer(val_questions, truncation=True, padding=True, return_tensors = 'pt')
val_context_encodings = passage_tokenizer(val_contexts, truncation=True, padding=True, return_tensors = 'pt')

# Model

In [9]:
class DPR(nn.Module):


  def __init__(self, query_model, passage_model, query_tokenizer, passage_tokenizer, 
              dense_size, freeze_params = 0.0, batch_size = 2, sample_size = 4):
    
    '''
    :query_model : The model that encodes queries to dense representation
    :passage_model : The model that encodes passages to dense representation
    :query_tokenizer : tokenizer for queries
    :passage_tokenizer : tokenizer for passages
    :passage_dict : dictionary of passages with their unique id
    :questions : A list of tuples with question and their correct passage id
    :dense_size : the dimension to which the DPR has to encode
    :freeze_params : the percentage of the parameters to be frozen
    :batch_size : the batch size for training
    :sample_size: the sample size for negative sampling
    '''
    super(DPR, self).__init__()
    self.query_model = query_model
    self.query_tokenizer = query_tokenizer
    self.passage_model = passage_model
    self.passage_tokenizer = passage_tokenizer
    self.freeze_params = freeze_params
    self.sample_size = sample_size
    self.batch_size = batch_size

    self.passage_to_dense = nn.Sequential(nn.Linear(768, dense_size * 2),
                                          nn.ReLU(),
                                          nn.Linear(dense_size * 2, dense_size),
                                          nn.GELU())
    
    self.query_to_dense = nn.Sequential(nn.Linear(768, dense_size * 2),
                                          nn.ReLU(),
                                          nn.Linear(dense_size * 2, dense_size),
                                          nn.GELU())
    self.log_softmax = nn.LogSoftmax(dim = 1)
    self.freeze_layers()


  # Freeze the first self.freeze_params % layers
  def freeze_layers(self):
    num_query_layers = sum(1 for _ in self.query_model.parameters())
    num_passage_layers = sum(1 for _ in self.passage_model.parameters())

    for parameters in list(self.query_model.parameters())[:int(self.freeze_params * num_query_layers)]:
      parameters.requires_grad = False

    for parameters in list(self.query_model.parameters())[int(self.freeze_params * num_query_layers):]:
      parameters.requires_grad = True

    for parameters in list(self.passage_model.parameters())[:int(self.freeze_params * num_passage_layers)]:
      parameters.requires_grad = False

    for parameters in list(self.passage_model.parameters())[int(self.freeze_params * num_passage_layers):]:
      parameters.requires_grad = True

  def get_passage_vectors(self, passage):
    p_vector = self.passage_model(input_ids = passage.input_ids, 
                                  attention_mask = passage.attention_mask)
    p_vector = self.query_to_dense(p_vector.pooler_output)
    return p_vector

  def get_query_vector(self, query):
    q_vector = self.query_model(input_ids = query.input_ids, 
                                attention_mask = query.attention_mask)
    q_vector = self.query_to_dense(q_vector.pooler_output)
    return q_vector

  def dot_product(self, q_vector, p_vector):
    q_vector = q_vector.unsqueeze(1)
    sim = torch.matmul(q_vector, torch.transpose(p_vector, -2, -1))
    return sim

  def forward(self, context_input_ids, context_attention_mask, query_input_ids, query_attention_mask):
    dense_passage = self.passage_model(input_ids = context_input_ids, attention_mask = context_attention_mask)
    dense_query = self.query_model(input_ids = query_input_ids, attention_mask = query_attention_mask)
    dense_passage = dense_passage['pooler_output']
    dense_query = dense_query['pooler_output']
    dense_passage = self.passage_to_dense(dense_passage)
    dense_query = self.query_to_dense(dense_query)
    similarity_score = self.dot_product(dense_query, dense_passage)
    similarity_score = similarity_score.squeeze(1)
    logits = self.log_softmax(similarity_score)
    return logits

In [10]:
dpr_model = DPR(query_model = query_model, 
                passage_model = passage_model, 
                query_tokenizer = query_tokenizer, 
                passage_tokenizer = passage_tokenizer, 
                dense_size = 64,
                freeze_params = 0.3,
                batch_size = 2,
                sample_size = 8)

dpr_model.train()


sum(p.numel() for p in dpr_model.parameters()), sum(p.numel() for p in dpr_model.parameters() if p.requires_grad == True)

(219177856, 125432704)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(dpr_model.parameters(), lr = 5e-5)

In [13]:
batch_size = 1
neg_samples = 2
num_questions = len(train_context_encodings.input_ids)

In [14]:
def get_batch():
  true = []
  context_input_ids_tensor = []
  context_attention_mask_tensor = []
  optimizer.zero_grad()

  true.append(1)

  ## Selecting the query and its positive context
  idx = random.randint(0, num_questions)
  context_input_ids = train_context_encodings.input_ids[idx]
  context_attention_mask = train_context_encodings.attention_mask[idx]
  query_input_ids = train_query_encodings.input_ids[idx]
  query_attention_mask = train_query_encodings.attention_mask[idx]
  context_input_ids_tensor.append(context_input_ids)
  context_attention_mask_tensor.append(context_attention_mask)

  ## Selecting the negative contexts which is hardcoded as 100 index from the selected positive context

  for j in range(neg_samples):
    true.append(0)
    neg_idx_1 = idx + 100
    neg_idx_2 = idx - 100
    if neg_idx_1 < num_questions:
      context_input_ids = train_context_encodings.input_ids[neg_idx_1]
      context_attention_mask = train_context_encodings.attention_mask[neg_idx_1]
      context_input_ids_tensor.append(context_input_ids)
      context_attention_mask_tensor.append(context_attention_mask)

    elif neg_idx_2 > 0:
      context_input_ids = train_context_encodings.input_ids[neg_idx_2]
      context_attention_mask = train_context_encodings.attention_mask[neg_idx_2]
      context_input_ids_tensor.append(context_input_ids)
      context_attention_mask_tensor.append(context_attention_mask)

  
  context_input_ids_tensor = torch.stack(context_input_ids_tensor)
  context_attention_mask_tensor = torch.stack(context_attention_mask_tensor)
  query_input_ids = query_input_ids.unsqueeze(0)
  query_attention_mask = query_attention_mask.unsqueeze(0)
  return context_input_ids_tensor, context_attention_mask_tensor, query_input_ids, query_attention_mask, true

In [15]:
batch_loss = 0
for i in range(1, 500):
  context_input_ids_tensor, context_attention_mask_tensor,query_input_ids,query_attention_mask, true = get_batch()
  pred = dpr_model(context_input_ids_tensor, context_attention_mask_tensor,query_input_ids,query_attention_mask)
  
  true = torch.tensor([0])
  loss = criterion(pred, true)
  loss.backward()
  batch_loss += loss.item()
  optimizer.step()
  if i%20 == 0:
    print(f"Batch : {int(i/20)}  Loss : {batch_loss/20}")
    batch_loss = 0

Batch : 1  Loss : 1.096655935049057
Batch : 2  Loss : 1.089974018931389
Batch : 3  Loss : 1.104052609205246
Batch : 4  Loss : 1.1004404127597809
Batch : 5  Loss : 1.1005908399820328
Batch : 6  Loss : 1.1046346187591554
Batch : 7  Loss : 1.1017967641353608
Batch : 8  Loss : 1.093809086084366
Batch : 9  Loss : 1.0951356291770935
Batch : 10  Loss : 1.0991297364234924
Batch : 11  Loss : 1.1008016765117645
Batch : 12  Loss : 1.1062632262706757
Batch : 13  Loss : 1.0976685404777526
Batch : 14  Loss : 1.1070590972900392
Batch : 15  Loss : 1.100395518541336
Batch : 16  Loss : 1.0968985676765441
Batch : 17  Loss : 1.1019813179969788
Batch : 18  Loss : 1.0935469359159469
Batch : 19  Loss : 1.1144321262836456
Batch : 20  Loss : 1.0970988929271699
Batch : 21  Loss : 1.1107907950878144
Batch : 22  Loss : 1.0947912335395813
Batch : 23  Loss : 1.0970601677894591
Batch : 24  Loss : 1.098900318145752


# Saving Model

In [16]:
torch.save(dpr_model, "/content/drive/MyDrive/end2_capstone/dpr_capstone_p2.pt")

In [17]:
torch.save(dpr_model.state_dict(), '/content/drive/MyDrive/end2_capstone/dpr_capstone_p2.bin')